In [1]:
from registration_framework import Registration
from optims.cmaes_optim import CMAES
from optims.pso_optim import PSO_optim
from optims.ppso_optim import PPSO_optim
from optims.ppso_optim_improved1 import PPSO_optim1, PPSO_optim2, PPSO_optim3, PPSO_optim1_1
from utils.tools import Tools
from utils.common_config import CommonConfig

# 进行图像匹配的操作
config = Tools.load_yaml_config("matched_config.yaml")
res_path = f"{config.data_save_path}/{config.record_id}"
file_name = f"{config.record_id}_config.yaml"
Tools.save_obj_yaml(res_path, file_name, config)



r_width: 78, r_height: 64
H_Refer: 1.0899114322385377
